In [5]:
import os
import numpy as np
import pandas as pd
import shutil
store = pd.HDFStore('data_store.h5')

In [6]:
class SplitImages:
    def __init__(self,class_name, classes, images):
        self.classes = classes
        self.class_name = class_name
        self.img_dir = os.getcwd()
        self.images = images
        self.img_count = {}

    def prepare(self):
        img_name = {}
        for _class in self.classes:
            os.makedirs(os.path.join(self.class_name, 'test', _class))
            os.makedirs(os.path.join(self.class_name, 'train', _class))
            os.makedirs(os.path.join(self.class_name, 'val', _class))
            self.img_count[_class] = 0
            img_name[_class] = []
        classes = []
        for _class in self.classes:
            classes.append(_class)
            classes.append(_class[0].upper() + _class[1:])
        for image in self.images:
            flag_multiple_classes = False
            flag_class_present = False
            current_class = ''
            for _class in classes:
                if _class in image:
                    if not flag_class_present:
                        flag_class_present = True
                        current_class = _class[0].lower() + _class[1:]
                    else:
                        flag_multiple_classes = True
            if flag_class_present == True and flag_multiple_classes == False:
                self.img_count[current_class] = self.img_count[current_class] + 1
                img_name[current_class].append(image)
        return img_name

    def split(self, img_name, train_test_split, train_validation_split, image_count, img_path):
        np.random.seed(0)
        random_imgs = {}
        for _class in self.classes:
            random_imgs_ids = np.random.choice(img_name[_class], image_count)
            random_imgs[_class] = random_imgs_ids

        train_count = int((1-train_test_split)*image_count)
        val_count = int(train_validation_split*train_count)
        test_img = {}
        train_img = {}
        val_img = {}
        
        for _class in self.classes:
            train_img[_class] = random_imgs[_class][ :train_count]
            test_img[_class]  = random_imgs[_class][train_count: ]
            train_img[_class] = train_img[_class][val_count: ]
            val_img[_class]   = train_img[_class][ :val_count]

        print('image_count:',image_count, 'train_count' , (train_count-val_count), 'test_count', 
                  (image_count-train_count), 'val_count' , val_count )    
        for _class in self.classes:
            count = 0
            for image in train_img[_class]:
                count = count+1
                shutil.copy2(image, img_path + '/' + self.class_name + '/' +
                             'train' + '/' + _class + '/' + str(count) + '.jpg')
        
            count = 0
            for image in val_img[_class]:
                count = count+1
                shutil.copy2(image, img_path + '/' + self.class_name + '/' +
                             'val' + '/' + _class + '/' + str(count) + '.jpg')
        
            count = 0
            for image in test_img[_class]:
                count = count+1
                shutil.copy2(image, img_path + '/' + self.class_name + '/' +
                             'test' + '/' + _class + '/' +  str(count) + '.jpg')

        return train_img, val_img, test_img
